<a href="https://colab.research.google.com/github/Dev-hottae/PINC/blob/master/Data_Analysis/Train_KoBERT_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls drive/'My Drive'/'Colab Notebooks'/

 __MACOSX	  'report.ipynb의 사본의 사본'	     requirements.txt
 NarrativeKoGPT2  'report.ipynb의 사본의 사본 (1)'   Untitled0.ipynb


In [3]:
!pip install -r drive/'My Drive'/'Colab Notebooks'/requirements.txt

     |████████████████████████████████| 348kB 4.7MB/s 
     |████████████████████████████████| 55.0MB 60kB/s 
     |████████████████████████████████| 1.1MB 59.5MB/s 
     |████████████████████████████████| 1.3MB 54.5MB/s 
     |████████████████████████████████| 890kB 51.1MB/s 
     |████████████████████████████████| 2.9MB 46.2MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588515 sha256=4906c9e3beb4afb5591b6a18400924445d021e83b4f588eaee8752445fb6a192
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251585 sha256=ec2b29217422b3b723105dc16c1025dab05edfba000a1e3f092529203dc414a7
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=ab2efde879a9fb988ce339f105c70e4f6

In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git

  Cloning https://****@github.com/SKTBrain/KoBERT.git to /tmp/pip-req-build-jtsx6fjn
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-jtsx6fjn
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12825 sha256=e63799a510416af2a43caa5a455d780189ae9e78a45cb52463e13a5df94f4024
  Stored in directory: /tmp/pip-ephem-wheel-cache-dbt55bbg/wheels/43/7b/50/91bc4d7bebb30b62f31a30a537787eb512f27506909162ba14
Successfully built kobert


In [52]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import gluonnlp as nlp
import numpy as np
import pandas as pd
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader
from torch import nn
from tqdm import tqdm, tqdm_notebook

In [7]:
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [8]:
#GPU 사용
device = torch.device("cuda:0")

# **SKT의 KoBERT 이용**

## **데이터 불러오기**

In [34]:
data_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/DataSet/sample_label_by1.csv'

In [35]:
news_data = pd.read_csv(data_path, encoding='utf-8', index_col=0)
news_data = news_data[['date', 'text', 'label']]
news_data = news_data.reset_index()
news_data = news_data.drop(['Unnamed: 0'], axis=1)
news_data

,date,text,label
0,2003-08-01,하이닉스 주가가 급반등했다. 1일 오후 2시23분 현재 하이닉스는 전날보다 1천1...,10
1,2003-08-01,거래소시장에서 종합지수가 美 경제지표의 호전에 고무된 외국인 매수로 연중 고점을 ...,10
2,2003-08-01,반도체주들이 강보합권에 개장했다. 1일 오전 9시5분 현재 삼성전자는 전날보다 3...,10
3,2003-08-01,1일 전기전자업종은 외국인 매수에 힘입어 삼성전자가 연중 고점을 경신하는 등 대부...,10
4,2003-08-04,반도체주들이 약세권에 개장했다. 4일 오전 9시5분 현재 삼성전자는 전날보다 5천...,1
...,...,...,...
15673,2020-09-28,코스피는 신종 코로나바이러스 감염증 재확산 우려에도 전일 미국 증시 상승 영향에 ...,10
15674,2020-09-28,코스피는 외국인과 기관의 매수에 오후 상승 폭을 키웠다. 28일 코스피는 오후 ...,10
15675,2020-09-29,코스피는 연휴를 앞두고 기관이 장중 순매수 기조를 이어간 영향으로 상승 마감했다....,10
15676,2020-09-29,코스피는 미국 추가 부양책 합의 기대로 상승하고 있다. 29일 코스피는 9시 6...,10


In [36]:
news_data = news_data[news_data['label']!=10]
news_data = news_data.reset_index()
news_data = news_data.drop(['index'], axis=1)
news_data.head()

,date,text,label
0,2003-08-04,반도체주들이 약세권에 개장했다. 4일 오전 9시5분 현재 삼성전자는 전날보다 5천...,1
1,2003-08-04,4일 전기전자업종은 주말 미국증시가 하락세를 보인 가운데 종목별로 혼조세를 보였다...,1
2,2003-08-06,미국시장의 고용지표 악화가 6일 오전 거래소시장에서 종합지수의 급락을 이끌고 있다...,0
3,2003-08-06,삼성전자가 사상최고치 돌파를 앞두고 번번이미끄러지고 있다. 6일 오전 9시45분 ...,0
4,2003-08-06,삼성전자 등 반도체주들이 오후장 들어 낙폭이 커졌다. 6일 오후 1시57분 현재 ...,0


In [37]:
# 학습, 테스트 데이터 분리
train_data = news_data.sample(frac=0.8, random_state=2020)
test_data = news_data.drop(train_data.index)
len(train_data), len(test_data)

(9254, 2313)

In [39]:
count_0 = 0
count_1 = 0
for i in tqdm(a):
    if i == 0:
        count_0 += 1
    else:
        count_1 += 1

count_0, count_1

100%|██████████| 9254/9254 [00:00<00:00, 908846.07it/s]


(4395, 4859)

In [40]:
count_0 / (count_0 + count_1)

0.47492976010373894

In [41]:
train_data = train_data.dropna()
train_data = train_data.reset_index()
train_data = train_data.drop(['index'], axis=1)
test_data = test_data.dropna()
test_data = test_data.reset_index()
test_data = test_data.drop(['index'], axis=1)
len(train_data), len(test_data)

(9254, 2313)

In [42]:
dataset_train = [] # 라벨은 0부터 순서대로 입력해야함
dataset_test = []
for i in tqdm(range(len(train_data))):
    dataset_train.append([train_data['text'][i], int(train_data['label'][i])]) # 해당 리스트 형태를 맞춰야 학습 가능
for i in tqdm(range(len(test_data))):
    dataset_test.append([test_data['text'][i], int(test_data['label'][i])])

dataset_test[:5], dataset_train[:5]

100%|██████████| 2313/2313 [00:00<00:00, 80089.04it/s]


([[' 외국인과 기관이 쌍끌이에 나서면서 지수가하락하루만에 오름세로 반전되면서 760선을 돌파했다. 27일 거래소시장에서 종합지수는 미국증시가 사흘만에 오름세로 반전되면서 강보합권에서 출발했다. 이번주들어 외국인의 매수강도가 주춤하고 있으나 지난주 후반 750선을 돌파한이후 이 지수대가 지지선으로 굳혀지면서 지수가 하방경직성을 보여줬다. 지수는 오전 11시 현재 1% 오른 760.56을 기록중이며 외국인이 202억원, 기관이143억원을 순매수중이다. 기관의 경우 지난주 일부 펀드의 해지에 따라 매도우위를 보였으나 이번주들어주식연계증권과 관련된 현물매수가 유입되고 있는 것으로 분석되고 있다. 업종별로 증권, 기계, 의료정밀, 통신, 은행 등의 시세탄력이 확대되고 있는 반면 섬유의복, 종이금속, 비금속 등은 약세권에 머물고 있다. 시가총액 상위종목은 삼성전자가 0.44% 내리면서 약보합권에 있으나 SK텔레콤, 국민은행, POSCO, KT, 하나은행 등은 오름세를 보이는 등 전반적으로 매수심리가 우세한 상황이다. 교보증권 이혜린연구원은 750선에서 지지력을 확인하면 하락리스크가 줄어든데다 미국증시의 반등에 심리적 안정감을 더해줬다 면서 그러나 여전히 주식형수익잔고나 고객예탁금 등 수급 측면의 부담이 지속되고 있어 해외요인에 의한 장세전개가계속될 것 이라고 전망했다. ',
   0],
  [' 가격부담이 현실화되면서 지수가 내림세로 돌아섰다. 28일 거래소시장에서 종합지수는 개장초 외국인 매수흐름이 이어지면서 760선을상회하는 강세로 출발했으나 시간이 흐를수록 기관과 개인의 매도물량 부담으로 약세권으로 돌아섰다. 장중 한때 749.34까지 떨어졌던 지수는 외국인의 순매수규모가 1천억원대를 넘어서며 추가하락이 주춤, 결국 0.81% 내린 752.81에 거래를 마쳤다. 외국인이 오후 3시4분 현재 1천458억원을 순매수했으나 기관과 개인은 1천121억원, 314억원을 각각 순매도, 여전히 보수적 시황관을 드러냈다. 오늘 발표된 7월 산업활동 동향이 시장 기대치 수준에 

## **데이터 토큰화**

In [60]:
ctx= 'cuda'
cachedir='~/kogpt2/'
pytorch_kobert = {
    'url': 'https://kobert.blob.core.windows.net/models/kobert/pytorch/pytorch_kobert_2439f391a6.params',
    'fname': 'pytorch_kobert_2439f391a6.params',
    'chksum': '2439f391a6'
}

bert_config = {
    'attention_probs_dropout_prob': 0.1,
    'hidden_act': 'gelu',
    'hidden_dropout_prob': 0.1,
    'hidden_size': 768,
    'initializer_range': 0.02,
    'intermediate_size': 3072,
    'max_position_embeddings': 512,
    'num_attention_heads': 12,
    'num_hidden_layers': 12,
    'type_vocab_size': 2,
    'vocab_size': 8002
}

In [63]:
# download model
model_info = pytorch_kobert
model_path = download(model_info['url'],
                      model_info['fname'],
                      model_info['chksum'],
                      cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                      vocab_info['fname'],
                      vocab_info['chksum'],
                      cachedir=cachedir)

NameError: ignored

In [59]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


RuntimeError: ignored

In [45]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


NameError: ignored

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## **모델링**

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2, # 해당 부분 파라미터 조정으로 다중 분류 가능
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total) # Warm-Up Step 추가

In [ ]:
def calc_accuracy(X,Y): # 정확도 계산 함수
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## **모델 학습**

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    # 모델 학습 : model.train()을 지정해줘야 Fine Tunning이 이뤄짐
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # 모델 평가 : model.eval()을 지정해야 Fine Tunning을 멈추고 평가를 시작함
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # 에폭마다 체크포인트 저장
    PATH = "/home/lab12/workspace/bigcontest/data_analysis/model_checkpoint/{}_epochs/".format(e+1) # 모델 저장 경로 설정
    torch.save(model, PATH + 'model.pt')  # 전체 모델 저장
    torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
    torch.save({
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict()
                }, PATH + 'all.tar')

epoch 2 batch id 1 loss 0.554951548576355 train acc 0.71875
epoch 2 batch id 201 loss 0.5218873620033264 train acc 0.7101212686567164
epoch 2 batch id 401 loss 0.45010921359062195 train acc 0.7061253117206983
epoch 2 batch id 601 loss 0.6309182643890381 train acc 0.7065307820299501
epoch 2 batch id 801 loss 0.5050973296165466 train acc 0.7078261548064919
epoch 2 batch id 1001 loss 0.5672126412391663 train acc 0.7094155844155844
epoch 2 batch id 1201 loss 0.5219614505767822 train acc 0.7114253746877602
epoch 2 batch id 1401 loss 0.5552719831466675 train acc 0.7125044610992148
epoch 2 batch id 1601 loss 0.4990692138671875 train acc 0.7138799968769519
epoch 2 batch id 1801 loss 0.5215151309967041 train acc 0.7158002498611882
epoch 2 batch id 2001 loss 0.42466288805007935 train acc 0.7175630934532734
epoch 2 batch id 2201 loss 0.567004382610321 train acc 0.7188067923671059
epoch 2 batch id 2401 loss 0.4554973542690277 train acc 0.7199344023323615
epoch 2 batch id 2601 loss 0.53680449724197


epoch 2 test acc 0.6849121476115897


epoch 3 batch id 1 loss 0.5583212375640869 train acc 0.71875
epoch 3 batch id 201 loss 0.5698355436325073 train acc 0.7060789800995025
epoch 3 batch id 401 loss 0.41047367453575134 train acc 0.7082294264339152
epoch 3 batch id 601 loss 0.6113224625587463 train acc 0.7093646006655574
epoch 3 batch id 801 loss 0.5261039137840271 train acc 0.7105376092384519
epoch 3 batch id 1001 loss 0.5396313071250916 train acc 0.7109921328671329
epoch 3 batch id 1201 loss 0.4963166415691376 train acc 0.7133248334721066
epoch 3 batch id 1401 loss 0.4880925416946411 train acc 0.7150361349036403
epoch 3 batch id 1601 loss 0.500450849533081 train acc 0.7165053091817614
epoch 3 batch id 1801 loss 0.5097231268882751 train acc 0.7186632426429761
epoch 3 batch id 2001 loss 0.4262673854827881 train acc 0.7210691529235382
epoch 3 batch id 2201 loss 0.6262375712394714 train acc 0.7230946160835984
epoch 3 batch id 2401 loss 0.4919337034225464 train acc 0.7249648583923365
epoch 3 batch id 2601 loss 0.51729017496109


epoch 3 test acc 0.6966400743931088


epoch 4 batch id 1 loss 0.37403547763824463 train acc 0.875
epoch 4 batch id 201 loss 0.3503040671348572 train acc 0.7706001243781094
epoch 4 batch id 401 loss 0.3806527554988861 train acc 0.7722880299251871
epoch 4 batch id 601 loss 0.5747219324111938 train acc 0.7731905158069884
epoch 4 batch id 801 loss 0.43526554107666016 train acc 0.7751638576779026
epoch 4 batch id 1001 loss 0.4218243956565857 train acc 0.7764110889110889
epoch 4 batch id 1201 loss 0.38353922963142395 train acc 0.7787650915903414
epoch 4 batch id 1401 loss 0.4645388424396515 train acc 0.7800901142041399
epoch 4 batch id 1601 loss 0.4076917767524719 train acc 0.7815427857589007
epoch 4 batch id 1801 loss 0.3772083818912506 train acc 0.7835490699611327
epoch 4 batch id 2001 loss 0.3051791787147522 train acc 0.7855369190404797
epoch 4 batch id 2201 loss 0.4458267092704773 train acc 0.7877527260336211
epoch 4 batch id 2401 loss 0.2519046366214752 train acc 0.7895017700957934


KeyboardInterrupt: 